# Advanced QuickStart
https://www.tensorflow.org/tutorials/quickstart/advanced

Start with imports

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Then load the data

In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


Generate train and test datasets

In [14]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Then create a new model object to represent the model for processing

In [16]:
class HubertModel(Model):
    def __init__(self):
        super(HubertModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10)

    def call(self, x):
        x = self.flatten(self.conv1(x))
        x = self.d2(self.d1(x))
        return x

model = HubertModel()

Add loss function and optimizer

In [17]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()


Select the metrics to train on

In [18]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


create method for model training

In [19]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


create method for testing

In [20]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


Train the model

In [21]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )


Epoch 1, Loss: 0.13467815518379211, Accuracy: 95.9383316040039, Test Loss: 0.0678497850894928, Test Accuracy: 97.81999969482422
Epoch 2, Loss: 0.044158224016427994, Accuracy: 98.62999725341797, Test Loss: 0.06324230879545212, Test Accuracy: 97.95999908447266
Epoch 3, Loss: 0.02515268512070179, Accuracy: 99.18000030517578, Test Loss: 0.05490493029356003, Test Accuracy: 98.27999877929688
Epoch 4, Loss: 0.014395324513316154, Accuracy: 99.49333190917969, Test Loss: 0.056576743721961975, Test Accuracy: 98.43000030517578
Epoch 5, Loss: 0.01116469968110323, Accuracy: 99.64833068847656, Test Loss: 0.06268542259931564, Test Accuracy: 98.43000030517578


In [15]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.uint8)>